# 神经网络层初始化的常见方法
*默认情况下，PyTorch会根据一个范围均匀地初始化权重和偏置矩阵， 这个范围是根据输入和输出维度计算出的。*
## 一、常见初始化函数
* 正态分布初始化
  ```
  torch.nn.init.normal_(tensor, mean=0, std=1) 使值服从正态分布N(mean, std)，默认值为0，1
  ```
* 均匀分布初始化
  ```
  torch.nn.init.uniform_(tensor, a=0, b=1) 使值服从均匀分布U(a,b)
  ```
* 常数初始化
  ```
  torch.nn.init.constant_(tensor, val) 使值为常数val nn.init.constant_(w, 0.3)
  ```
* Xavier初始化
  


In [7]:
from torch import nn
net = nn.Sequential(nn.Linear(2, 1))

* 逐层初始化

In [8]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

* 整体初始化

In [9]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal) #所有层依次初始化
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 0.0144, -0.0082]), tensor(0.))

## 二、参数访问
* 索引访问

In [1]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

e:\coding_setting\Miniconda\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tensor([[0.2166],
        [0.1750]], grad_fn=<AddmmBackward0>)

In [3]:
net[0].state_dict()
# 其中偏置为 1*8，也就是会发生广播机制

OrderedDict([('weight',
              tensor([[-0.1198, -0.4384, -0.4139, -0.4539],
                      [-0.3994,  0.1911,  0.0418, -0.1442],
                      [ 0.0037, -0.1128,  0.3376, -0.0439],
                      [ 0.1721, -0.3166, -0.1561,  0.4517],
                      [ 0.2120, -0.4205,  0.4387,  0.1055],
                      [-0.4208, -0.4719,  0.4014, -0.2797],
                      [ 0.1947, -0.2509,  0.3885,  0.0649],
                      [ 0.0310, -0.2202, -0.0770, -0.3654]])),
             ('bias',
              tensor([-0.2739,  0.0335, -0.2795,  0.1677, -0.2742,  0.4082, -0.3677,  0.1699]))])

In [4]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.1530], requires_grad=True)
tensor([0.1530])


* 访问所有参数

In [6]:
[(name,param.shape) for name ,param in net[0].named_parameters()]

[('weight', torch.Size([8, 4])), ('bias', torch.Size([8]))]

## 三、参数绑定

In [10]:
# 我们需要给共享层一个名称，以便可以引用它的参数
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象，而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


**注意到第二层和第四层实际上共享了同一块数据，也就是更改其中一个层的数据另一个层也会改变**

### 共享参数的好处：
* 对于图像识别中的CNN，共享参数使网络能够在图像中的任何地方而不是仅在某个区域中查找给定的功能。
* 对于RNN，它在序列的各个时间步之间共享参数，因此可以很好地推广到不同序列长度的示例。
* 对于自动编码器，编码器和解码器共享参数。 在具有线性激活的单层自动编码器中，共享权重会在权重矩阵的不同隐藏层之间强制正交。

In [1]:
import torch
from torch import nn

torch.cuda.device_count()

e:\coding_setting\Miniconda\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1